# NB-IoT Localization - WKNN

## Data loading

In [20]:
import numpy as np
import pandas as pd

from scripts.weighted_coverage import create_point_matrix

rf_param = 'NSINR'

# Sample data for Reference Points (RPs)
data_rp = {
    'Latitude': [52.5200, 48.8566, 51.5074, 40.7128, 34.0522],
    'Longitude': [13.4050, 2.3522, -0.1278, -74.0060, -118.2437],
    'NPCI1': [1, 1, 2, 1, 2],
    'NPCI2': [2, 3, 2, 2, 3],
    'NPCI3': [8, 8, 8, 8, 8],
    'NSINR': [30, np.nan, 40, 25, 35]
}

# Sample data for Test Points (TPs)
data_tp = {
    'Latitude': [52.5200, 48.8566, 51.5074, 40.7128, 34.0522],
    'Longitude': [13.4050, 2.3522, -0.1278, -74.0060, -118.2437],
    'NPCI1': [1, 1, 2, 1, 2],
    'NPCI2': [2, 3, 2, 2, 3],
    'NPCI3': [8, 8, 8, 8, 8],
    'NSINR': [28, 22, 38, 20, 32]
}

# Create DataFrames
df_rp = pd.DataFrame(data_rp)
df_tp = pd.DataFrame(data_tp)

## Data processing

In [21]:
# Concatenate NPCI columns from both DataFrames
npcis_rp = pd.concat([df_rp['NPCI1'], df_rp['NPCI2'], df_rp['NPCI3']])
npcis_tp = pd.concat([df_tp['NPCI1'], df_tp['NPCI2'], df_tp['NPCI3']])

all_npcis = pd.concat([npcis_rp, npcis_tp])
unique_npcis = np.unique(all_npcis)

# Create matrices for reference points
m_rfp, idx_rfp = create_point_matrix(df_rp, unique_npcis, rf_param)

# Create matrices for test points 
m_tp, idx_tp = create_point_matrix(df_tp, unique_npcis, rf_param)


## Compute weights

In [22]:
from scipy.spatial.distance import cdist

# Caclulate distances between tps and rps
D = cdist(m_tp, m_rfp, metric='euclidean')

# Normalize distances based on common NPCIs
for i in range(m_tp.shape[0]):
    match = np.logical_and(idx_tp[i, :], idx_rfp)
    s = np.sum(match, axis=1)
    z = np.where(s == 0)
    nz = np.where(s != 0)

    for j in nz[0]:
        D[i, j] = D[i, j] / s[j]
    for j in z[0]:
        D[i, j] = np.inf  # Use np.inf to represent a very large distance

# Set distances to dummy reference points to a very large value
dummy_rfps = np.all(idx_rfp == 0, axis=1)
D[:, dummy_rfps] = np.inf

# Replace zero distances with a small value to avoid singularities
D[D == 0] = np.min(D[D != 0]) / 20

# Sort distances and compute weights
D_sort = np.sort(D, axis=1)
idx_sort = np.argsort(D, axis=1)
W = 1.0 / D_sort

## WKNN

In [23]:
from scripts.haversine import haversine_distance

num_tps = df_tp.shape[0]
k_max = 40
k_values = range(1, k_max + 1)
TP_est_location = [None] * len(k_values)
avg_errors = {}

# Extract real positions of test points
real_lat = df_tp['Latitude'].values
real_long = df_tp['Longitude'].values
real_position = np.vstack((real_lat, real_long)).T

# Loop over each k value
for i, this_k in enumerate(k_values):
    # Select the k-nearest reference points
    RFP_selected_idx = idx_sort[:, :this_k]

    # Extract coordinates of the selected reference points
    lat_k_RFP_matrix = df_rp.iloc[RFP_selected_idx.flatten()]['Latitude'].values.reshape(RFP_selected_idx.shape)
    long_k_RFP_matrix = df_rp.iloc[RFP_selected_idx.flatten()]['Longitude'].values.reshape(RFP_selected_idx.shape)

    # Compute weighted sums of coordinates
    sum_lat = np.sum(lat_k_RFP_matrix * W[:, :this_k], axis=1)
    sum_long = np.sum(long_k_RFP_matrix * W[:, :this_k], axis=1)

    # Compute estimated coordinates of test points
    lat_k_TP = sum_lat / np.sum(W[:, :this_k], axis=1)
    long_k_TP = sum_long / np.sum(W[:, :this_k], axis=1)

    # Compute errors using Haversine formula
    km_pow = haversine_distance(real_position[:, 0], real_position[:, 1], lat_k_TP, long_k_TP)
    average_error_pow = np.mean(km_pow)

    avg_errors[this_k] = average_error_pow

    # Store estimated locations
    TP_est_location_k = np.zeros((num_tps, 2))
    TP_est_location_k[:, 0] = lat_k_TP
    TP_est_location_k[:, 1] = long_k_TP
    TP_est_location[i] = TP_est_location_k